<a href="https://colab.research.google.com/github/swankystark/Google-Tunix-Hack/blob/main/Reference_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reference Program**

In [ ]:
def show_hbm_usage():
  """Displays memory usage per device."""
  fmt_size = functools.partial(humanize.naturalsize, binary=True)

  for d in jax.local_devices():
    stats = d.memory_stats()
    used = stats["bytes_in_use"]
    limit = stats["bytes_limit"]
    print(f"Using {fmt_size(used)} / {fmt_size(limit)} ({used/limit:%}) on {d}")

This is useful because:

>Fine-tuning Gemma + LoRA + GRPO is memory heavy.

> If memory usage exceeds limits → TPU crashes.

> This function lets you check memory usage anytime during the notebook.

In [ ]:
for ele in train_dataset[:1]:
  pprint(ele)


It is a sanity check to confirm:

*   the template is correct

*   prompts look correct


*   answer extraction worked

*   the dataset pipeline is producing the right format





In [ ]:
!rm /tmp/content/intermediate_ckpt/* -rf

!rm /tmp/content/ckpts/* -rf

if model_family == "gemma2":
  params = params_lib.load_and_format_params(
      os.path.join(kaggle_ckpt_path, "gemma2-2b-it")
  )
  gemma = gemma_lib.Transformer.from_params(params, version="2-2b-it")
  checkpointer = ocp.StandardCheckpointer()
  _, state = nnx.split(gemma)
  checkpointer.save(os.path.join(INTERMEDIATE_CKPT_DIR, "state"), state)
  checkpointer.wait_until_finished()
  del params
  del gemma
  del state
  gc.collect()


A checkpoint is a saved version of your model’s weights (parameters) at a specific moment during training.


1.   Before training → load checkpoint of pretrained Gemma

2.   During training → save checkpoints every few steps

3.   After training → final checkpoint contains the tuned model



In [ ]:
# V1: TPU Check
print(f"JAX devices: {jax.devices()}")
assert jax.device_count() > 0, "No TPU found!"


jax.devices() returns a list of all available compute devices: CPU, GPU, TPU.

> Printing it helps you verify: “Am I really on a TPU notebook / Colab / Kaggle with TPU?”

jax.device_count() gives how many devices there are.